## Ejemplo análisis de una sección

In [1]:
from mento import Concrete_ACI_318_19, SteelBar, RectangularBeam
from mento import cm, MPa

conc= Concrete_ACI_318_19(name="C25",f_c=25*MPa) 
steel= SteelBar(name="ADN 420", f_y=420*MPa) 
beam = RectangularBeam(label="101",concrete=conc,steel_bar=steel,width=20*cm, height=40*cm)
beam.data

Beam 101, $b$=20.00 cm, $h$=40.00 cm, $c_{c}$=2.50 cm,                             Concrete C25, Rebar ADN 420.

In [2]:
from mento import Concrete_ACI_318_19, SteelBar, Forces, RectangularBeam, Node
from mento import mm, cm, kN, MPa, kNm
# Defines materiales and beam section
conc= Concrete_ACI_318_19(name="C25",f_c=25*MPa) 
steel= SteelBar(name="ADN 420", f_y=420*MPa) 
beam = RectangularBeam(label="101",concrete=conc,steel_bar=steel,width=20*cm, height=40*cm)
# Define forces
f1 = Forces(label='1.4D', V_z=50*kN, M_y=90*kNm)
f2 = Forces(label='1.2D+1.6L', V_z=155*kN, M_y=-80*kNm)
f3 = Forces(label='W', V_z=220*kN)
f4 = Forces(label='S', V_z=80*kN)
f5 = Forces(label='E', V_z=10*kN)
# Create node and assign beam section and list of forces
Node(section=beam, forces=[f1, f2, f3, f4, f5])
# Assign transverse reiforcement
beam.set_transverse_rebar(n_stirrups=1, d_b=8*mm, s_l=20*cm)
beam.set_longitudinal_rebar_bot(n1=2,d_b1=16*mm, n2=1, d_b2=12*mm, n3=2,d_b3=12*mm, n4=1, d_b4=10*mm)
beam.set_longitudinal_rebar_top(n1=2,d_b1=16*mm)
# Check shear for all forces
beam.check_shear()
beam.check_flexure()
# print(beam.check_shear())
beam.results
# beam.design_shear()
# beam.results
# beam.shear_results_detailed()

Beam 101, $b$=20.00 cm, $h$=40.00 cm, $c_{c}$=2.50 cm,                             Concrete C25, Rebar ADN 420.

Top longitudinal rebar: 2Ø16, $A_{s,top}$ = 4.02 cm², $M_u$ = -80 kNm, $\phi M_n$ = 51.55 kNm, $\color{#d43e36}{\text{DCR}=1.55}$

Bottom longitudinal rebar: 2Ø16+1Ø12 ++ 2Ø12+1Ø10, $A_{s,bot}$ = 8.2 cm², $M_u$ = 90 kNm, $\phi M_n$ = 93.58 kNm, $\color{#efc200}{\text{DCR}=0.96}$

AttributeError: 'RectangularBeam' object has no attribute '_all_shear_checks_passed'

In [5]:
from mento.summary import BeamSummary
import pandas as pd

conc = Concrete_ACI_318_19(name="C25", f_c=25*MPa)
steel = SteelBar(name="ADN 420", f_y=420*MPa)
input_df = pd.read_excel('Mento-Input.xlsx', sheet_name='Beams', usecols='B:R', skiprows=4)
# data = {'Label': ['', 'V101', 'V102', 'V103', 'V104'],
#         'b': ['cm', 20, 20, 20, 20],
#         'h': ['cm', 50, 50, 50, 50],
#         'Nx': ['kN', 0, 0, 0, 50],
#         'Vz': ['kN', 20, 50, 100, 100],
#         'My': ['kNm', 0, 35, 40, 45],
#         'ns': ['', 0, 1.0, 1.0, 1.0],
#         'dbs': ['mm', 0, 6, 6, 6],
#         'sl': ['cm', 0, 20, 20, 20],
#         'n1': ['', 2.0, 2.0, 2.0, 2.0],
#         'db1': ['mm', 12, 12, 12, 12],
#         'n2': ['', 1.0, 0.0, 1.0, 0.0],
#         'db2': ['mm', 10, 0, 10, 0],
#         'n3': ['', 2.0, 0.0, 2.0, 0.0],
#         'db3': ['mm', 12, 0, 0, 0],
#         'n4': ['', 1.0, 0.0, 1.0, 0.0],
#         'db4': ['mm', 10, 0, 0, 0]}
# input_df = pd.DataFrame(data)
# print(input_df)
beam_summary = BeamSummary(concrete=conc, steel_bar=steel, beam_list=input_df)
# beam_summary.data
beam_summary.check(capacity_check=True)
beam_summary.check()
# beam_summary.check().to_excel('hola.xlsx', index=False)
beam_summary.shear_results(capacity=False)
# beam_summary.shear_results(capacity=True)
# print(results)
# print(beam_summary.beams[3]['section'].shear_results_detailed)

In [ ]:
from pint import UnitRegistry

# Initialize a UnitRegistry for handling units with Pint
ureg = UnitRegistry()
PlainQuantity = ureg.Quantity

def layer_clear_spacing(n_a: int, d_a: PlainQuantity, n_b: int, d_b: PlainQuantity, width: PlainQuantity, c_c: PlainQuantity, stirrup_d_b: PlainQuantity) -> PlainQuantity:
    """
    Helper function to calculate clear spacing for a given layer.

    Parameters:
        n_a (int): Number of bars in the first group of the layer.
        d_a (PlainQuantity): Diameter of bars in the first group of the layer.
        n_b (int): Number of bars in the second group of the layer.
        d_b (PlainQuantity): Diameter of bars in the second group of the layer.
        width (PlainQuantity): Width of the section.
        c_c (PlainQuantity): Clear cover.
        stirrup_d_b (PlainQuantity): Diameter of the stirrup bars.

    Returns:
        PlainQuantity: Clear spacing for the given layer.
    """
    effective_width = width - 2 * (c_c + stirrup_d_b)
    total_bars = n_a + n_b
    if total_bars <= 1:
        return effective_width - max(d_a, d_b)  # Clear space for one bar
    total_bar_width = n_a * d_a + n_b * d_b
    return (effective_width - total_bar_width) / (total_bars - 1)

# Example usage with dummy values
n1, d_b1 = 2, 20 * ureg.mm  # Number of bars and diameter in group 1
n2, d_b2 = 1, 12 * ureg.mm  # Number of bars and diameter in group 2
n3, d_b3 = 4, 12 * ureg.mm  # Number of bars and diameter in group 3
n4, d_b4 = 3, 16 * ureg.mm  # Number of bars and diameter in group 4

width = 200 * ureg.mm  # Section width
c_c = 25 * ureg.mm     # Clear cover
stirrup_d_b = 10 * ureg.mm  # Diameter of stirrup bars

# Calculate clear spacing for each layer
spacing_layer1 = layer_clear_spacing(n1, d_b1, n2, d_b2, width, c_c, stirrup_d_b)
spacing_layer2 = layer_clear_spacing(n3, d_b3, n4, d_b4, width, c_c, stirrup_d_b)

print(f"Clear spacing for layer 1: {spacing_layer1:.2f}")
print(f"Clear spacing for layer 2: {spacing_layer2:.2f}")
